# Cleaning & Vectorizing Text

In [292]:
import pandas as pd
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, roc_curve
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import AdaBoostClassifier


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [253]:
#importing datasets
training = pd.read_csv('Training Data.csv', encoding='UTF-8')
training = training.dropna()
testing= pd.read_csv('Testing Data.csv')
testing = testing.dropna()

#changing binary values from float to integer
training['label'] = training['label'].astype(int)
testing['Spam'] = testing['Spam'].astype(int)

In [342]:
training['email'][0]

'date wed NUMBER aug NUMBER NUMBER NUMBER NUMBER NUMBER chris garrigues cwg dated NUMBER NUMBERfaNUMBERd deepeddy com message id NUMBER NUMBER tmda deepeddy vircio com reproduce error repeatable like every time without fail debug log pick happening NUMBER NUMBER NUMBER pick exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER ftoc pickmsgs NUMBER hit NUMBER NUMBER NUMBER marking NUMBER hits NUMBER NUMBER NUMBER tkerror syntax error expression int note run pick command hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER hit NUMBER hit comes obviously version nmh using delta pick version pick nmh NUMBER NUMBER NUMBER compiled URL sun mar NUMBER NUMBER NUMBER NUMBER ict NUMBER relevant part mh profile delta mhparam pick seq sel list since pick command works sequence 

In [255]:
#Text Cleaning

def text_cleaning(i):
    func_list = []
    for text in i:
        #text = text.lower()
        text = re.sub('[^A-Za-z0-9]+', ' ', text)
        func_list.append(text)
    return func_list

In [256]:
training['email'] = text_cleaning(training['email'])

In [257]:
for i in range(len(training['email'])):
    
    temp=[]
    
    for w in training['email'][i].split():
        if w.lower() not in stopwords.words('english'):
                temp.append(w)

    training['email'][i] = ' '.join(temp)


C:\Users\dvazq\Anaconda3\envs\DataMining\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [258]:
training['email']

0       date wed NUMBER aug NUMBER NUMBER NUMBER NUMBE...
1       martin posted tassos papadopoulos greek sculpt...
2       man threatens explosion moscow thursday august...
3       klez virus die already prolific virus ever kle...
4       adding cream spaghetti carbonara effect pasta ...
                              ...                        
2970    abc good morning america ranks NUMBER christma...
2971    hyperlink hyperlink hyperlink let mortgage len...
2972    thank shopping us gifts occasions free gift NU...
2973    famous ebay marketing e course learn sell comp...
2974    hello chinese traditional NUMBER NUMBER f r v ...
Name: email, Length: 2975, dtype: object

In [197]:
# word_bag=[]

# for s in training['email']:
#     for w in s:
#         word_bag.append(w)

In [259]:
#applying countvectorizer

vectorizer = TfidfVectorizer(TfidfVectorizer(ngram_range=(1,1),strip_accents='unicode', lowercase=True, 
                                             analyzer='word',stop_words='english')
)
tfidf = vectorizer.fit(training['email'])

C:\Users\dvazq\Anaconda3\envs\DataMining\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass input=TfidfVectorizer(stop_words='english', strip_accents='unicode') as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


# Splitting the Data

In [263]:
x = training['email']
y = training['label']

In [264]:
x

0       date wed NUMBER aug NUMBER NUMBER NUMBER NUMBE...
1       martin posted tassos papadopoulos greek sculpt...
2       man threatens explosion moscow thursday august...
3       klez virus die already prolific virus ever kle...
4       adding cream spaghetti carbonara effect pasta ...
                              ...                        
2970    abc good morning america ranks NUMBER christma...
2971    hyperlink hyperlink hyperlink let mortgage len...
2972    thank shopping us gifts occasions free gift NU...
2973    famous ebay marketing e course learn sell comp...
2974    hello chinese traditional NUMBER NUMBER f r v ...
Name: email, Length: 2975, dtype: object

In [265]:
X_train, X_test, y_train, y_test = train_test_split(tfidf.fit_transform(x), y, test_size=0.2)

In [266]:
y_train

525     0
2875    1
1695    0
2823    1
1687    0
       ..
1707    0
2905    1
210     0
785     0
1129    0
Name: label, Length: 2380, dtype: int32

# Logistic Regression

In [334]:
logmodel = LogisticRegression(C=25)
#build the model
log_mod =logmodel.fit(X_train,y_train)

In [335]:
cross_model= cross_val_score(log_mod,X_train, y_train, cv=3, scoring= "f1_macro")
display(cross_model)
print("F1: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

cross_model= cross_val_score(log_mod,X_train, y_train, cv=3, scoring= "accuracy")
display(cross_model)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))

cross_model= cross_val_score(log_mod,X_train, y_train, cv=3, scoring= "precision")
display(cross_model)
print("Precision: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

#Confusion Matrix
print(confusion_matrix(y_test,log_mod.predict(X_test)))

array([0.94792418, 0.95287336, 0.95581352])

F1: 0.95 (+/- 0.01)


array([0.97355164, 0.97604035, 0.97730139])

Accuracy: 0.98 (+/- 0.00)


array([1., 1., 1.])

Precision: 1.00 (+/- 0.00)
[[493   0]
 [  8  94]]


# Decision Tree

In [336]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

In [337]:
cross_model= cross_val_score(clf,X_train, y_train, cv=3, scoring= "f1_macro")
display(cross_model)
print("F1: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

cross_model= cross_val_score(log_mod,X_train, y_train, cv=3, scoring= "accuracy")
display(cross_model)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))

cross_model= cross_val_score(clf,X_train, y_train, cv=3, scoring= "precision")
display(cross_model)
print("Precision: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

#Confusion Matrix
print(confusion_matrix(y_test,clf.predict(X_test)))

array([0.91822665, 0.91519194, 0.91204936])

F1: 0.92 (+/- 0.01)


array([0.97355164, 0.97604035, 0.97730139])

Accuracy: 0.98 (+/- 0.00)


array([0.90677966, 0.82394366, 0.84732824])

Precision: 0.86 (+/- 0.07)
[[479  14]
 [  6  96]]


# Random Forest with AdaBoost

In [338]:
rfc = RandomForestClassifier(n_estimators=50, criterion='entropy', bootstrap=False)
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=50, base_estimator=rfc,
                         learning_rate=1)

RDAB = abc.fit(X_train, y_train)

In [339]:
cross_model= cross_val_score(RDAB,X_train, y_train, cv=3, scoring= "f1_macro")
display(cross_model)
print("F1: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

cross_model= cross_val_score(log_mod,X_train, y_train, cv=3, scoring= "accuracy")
display(cross_model)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))

cross_model= cross_val_score(RDAB,X_train, y_train, cv=3, scoring= "precision")
display(cross_model)
print("Precision: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

#Confusion Matrix
print(confusion_matrix(y_test,RDAB.predict(X_test)))

array([0.95057579, 0.95550832, 0.92880595])

F1: 0.94 (+/- 0.02)


array([0.97355164, 0.97604035, 0.97730139])

Accuracy: 0.98 (+/- 0.00)


array([1., 1., 1.])

Precision: 1.00 (+/- 0.00)
[[493   0]
 [ 11  91]]


# Naive Bayes with AdaBoost

In [352]:
NB = MultinomialNB(alpha=0.0001, fit_prior=False)

abnb = AdaBoostClassifier(n_estimators=50, base_estimator=NB,
                         learning_rate=1)

ABNB = abnb.fit(X_train, y_train)

In [341]:
cross_model= cross_val_score(ABNB,X_train, y_train, cv=3, scoring= "f1_macro")
display(cross_model)
print("F1: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

cross_model= cross_val_score(log_mod,X_train, y_train, cv=3, scoring= "accuracy")
display(cross_model)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))

cross_model= cross_val_score(ABNB,X_train, y_train, cv=3, scoring= "precision")
display(cross_model)
print("Precision: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

#Confusion Matrix
print(confusion_matrix(y_test,ABNB.predict(X_test)))

array([0.96297721, 0.97289815, 0.97222611])

F1: 0.97 (+/- 0.01)


array([0.97355164, 0.97604035, 0.97730139])

Accuracy: 0.98 (+/- 0.00)


array([0.9379845 , 0.92028986, 0.95348837])

Precision: 0.94 (+/- 0.03)
[[490   3]
 [  1 101]]


# Conclusion

In terms of accuracy all models scored 98%. However, in terms of precision, the logistic regression and AdaBoost with Random Forest as classifier scores a perfect score (100%). 

I also like the F1 score, which is a balance of the recall and precision score. The mode with the highest F1 score was the AdaBoost Classifier with Naive Bayes (97%). 

I would choose the AdaBoost Classifier with Naive Bayes as the best model for this dataset. I will now apply this classifier to my validation dataset which include my own personal emails. 

In [343]:
tfidf_testing = TfidfVectorizer(ngram_range=(1,1),strip_accents='unicode', lowercase=True, 
                         analyzer='word',stop_words='english',
                         vocabulary=tfidf.get_feature_names())

In [344]:
testing

,Email,Spam
0,"El MiÃ©rcoles, 14 de mayo, 2014 4:07 P.M., ""Al...",0
1,"El , David Vazquez Lizama <dvazquez_lizama@yah...",0
2,"Delta Fligh Numer - Just call to change, they ...",0
3,"Hi President Peterson,\nI hope you remember me...",0
4,"Thank you, President.\n\n~ David Vazquez\nEl J...",0
...,...,...
97,senior advocate of nigeria barr williams falan...,1
98,leasing deal of the year mercedes benz eNUMBE...,1
99,NUMBER lose NUMBER NUMBERlbs in NUMBER weeks f...,1
100,need health insurance in addition to featuring...,1


In [345]:
#Text Cleaning

def text_cleaning(i):
    func_list = []
    for text in i:
        #text = text.lower()
        text = re.sub('[^A-Za-z0-9]+', ' ', text)
        func_list.append(text)
    return func_list

In [346]:
testing['Email'] = text_cleaning(testing['Email'])

In [347]:
#Removing stop words

for i in range(len(testing['Email'])):
    
    temp=[]
    
    for w in testing['Email'][i].split():
        if w.lower() not in stopwords.words('english'):
                temp.append(w)

    testing['Email'][i] = ' '.join(temp)


C:\Users\dvazq\Anaconda3\envs\DataMining\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [357]:
#Mix around

testing = testing.sample(frac=1).reset_index(drop=True)

In [358]:
testing

,Email,Spam
0,becky adams snow edu David Vazquez,0
1,Hi Julia nice see Kelly Thank taking time come...,0
2,hello jm spamassassin taint orghuman growth ho...,1
3,Hi Dean actual leaving country today aware wou...,0
4,Hi Barbara Tomorrow could go morning around 10...,0
...,...,...
97,ya se acabo el norton que tarjeta utilizamos p...,0
98,Hi Becky e mail address wanted know placement ...,0
99,Thank President David Vazquez El Jueves 22 de ...,0
100,office egnr femi daniel federal ministry works...,1


In [359]:
test_x = testing['Email']
test_y = testing['Spam']

In [360]:
NB = MultinomialNB(alpha=0.0001, fit_prior=False)

abnb = AdaBoostClassifier(n_estimators=50, base_estimator=NB,
                         learning_rate=1)

ABNB.predict(tfidf_testing.fit_transform(test_x))

array([0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0])

In [363]:
cross_model= cross_val_score(ABNB,tfidf_testing.fit_transform(test_x), test_y, scoring= "f1_macro")
display(cross_model)
print("F1: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

cross_model= cross_val_score(ABNB,tfidf_testing.fit_transform(test_x), test_y, cv=3, scoring= "accuracy")
display(cross_model)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))

cross_model= cross_val_score(ABNB,tfidf_testing.fit_transform(test_x), test_y, cv=3, scoring= "precision")
display(cross_model)
print("Precision: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

#Confusion Matrix
print(confusion_matrix(test_y,ABNB.predict(tfidf_testing.fit_transform(test_x))))

array([1.        , 1.        , 1.        , 0.8989899 , 0.94666667])

F1: 0.97 (+/- 0.08)


array([1.        , 0.91176471, 0.97058824])

Accuracy: 0.96 (+/- 0.07)


array([1.        , 0.82352941, 1.        ])

Precision: 0.94 (+/- 0.17)
[[55  6]
 [ 1 40]]


# Results
We can see that our model worked pretty well with all scores: F1, Accuracy, Precision. 

# Trying with the other 3 classifiers

In [365]:
#Logistic Regression

log_mod.predict(tfidf_testing.fit_transform(test_x))

cross_model= cross_val_score(log_mod,tfidf_testing.fit_transform(test_x), test_y, scoring= "f1_macro")
display(cross_model)
print("F1: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

cross_model= cross_val_score(log_mod,tfidf_testing.fit_transform(test_x), test_y, cv=3, scoring= "accuracy")
display(cross_model)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))

cross_model= cross_val_score(log_mod,tfidf_testing.fit_transform(test_x), test_y, cv=3, scoring= "precision")
display(cross_model)
print("Precision: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

#Confusion Matrix
print(confusion_matrix(test_y,log_mod.predict(tfidf_testing.fit_transform(test_x))))

array([1.        , 1.        , 1.        , 1.        , 0.82905983])

F1: 0.97 (+/- 0.14)


array([1.        , 1.        , 0.88235294])

Accuracy: 0.96 (+/- 0.11)


array([1., 1., 1.])

Precision: 1.00 (+/- 0.00)
[[60  1]
 [ 1 40]]


In [366]:
#Decision Tree

clf.predict(tfidf_testing.fit_transform(test_x))

cross_model= cross_val_score(clf,tfidf_testing.fit_transform(test_x), test_y, scoring= "f1_macro")
display(cross_model)
print("F1: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

cross_model= cross_val_score(clf,tfidf_testing.fit_transform(test_x), test_y, cv=3, scoring= "accuracy")
display(cross_model)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))

cross_model= cross_val_score(clf,tfidf_testing.fit_transform(test_x), test_y, cv=3, scoring= "precision")
display(cross_model)
print("Precision: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

#Confusion Matrix
print(confusion_matrix(test_y,clf.predict(tfidf_testing.fit_transform(test_x))))

array([0.95058824, 0.95194508, 0.9488491 , 0.9488491 , 1.        ])

F1: 0.96 (+/- 0.04)


array([0.94117647, 0.91176471, 0.97058824])

Accuracy: 0.94 (+/- 0.05)


array([0.92857143, 0.82352941, 1.        ])

Precision: 0.92 (+/- 0.14)
[[52  9]
 [ 1 40]]


In [367]:
#AdaBoost with Random Forest

RDAB.predict(tfidf_testing.fit_transform(test_x))

cross_model= cross_val_score(RDAB,tfidf_testing.fit_transform(test_x), test_y, scoring= "f1_macro")
display(cross_model)
print("F1: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

cross_model= cross_val_score(RDAB,tfidf_testing.fit_transform(test_x), test_y, cv=3, scoring= "accuracy")
display(cross_model)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))

cross_model= cross_val_score(RDAB,tfidf_testing.fit_transform(test_x), test_y, cv=3, scoring= "precision")
display(cross_model)
print("Precision: %0.2f (+/- %0.2f)" % (cross_model.mean(), cross_model.std() * 2))
#0.7 or more is good

#Confusion Matrix
print(confusion_matrix(test_y,RDAB.predict(tfidf_testing.fit_transform(test_x))))

array([1.        , 0.89903846, 1.        , 0.89010989, 0.82905983])

F1: 0.92 (+/- 0.13)


array([0.94117647, 1.        , 0.94117647])

Accuracy: 0.96 (+/- 0.06)


array([1., 1., 1.])

Precision: 1.00 (+/- 0.00)
[[61  0]
 [ 1 40]]


# They all work pretty well (F1 scores of 0.90+). 